<a href="https://colab.research.google.com/github/BruceXavierChou/stock_analysis/blob/main/%E3%80%8Cstk_ch03_ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CH-03 股市資料蒐集、爬蟲與搭建資料庫

## 3-2 資料爬蟲

## 證交所資料爬蟲
1. 進入證交所網址：https://www.twse.com.tw/zh/index.html
2. 使用開發者模式取得請求資料網址

### 1️⃣ 匯入套件

In [ ]:
import requests
import pandas as pd
import datetime as dt # 時間套件
from dateutil.relativedelta import relativedelta

### 2️⃣ 取得個股日成交資訊

In [ ]:
# 輸入股票代號
stock_id = '2330'
# 當日時間
date = dt.date.today().strftime("%Y%m%d")
# 取得證交所網站資料
stock_data = requests.get(f'https://www.twse.com.tw/rwd/zh/ \
            afterTrading/STOCK_DAY?date={date}&stockNo={stock_id}')
json_data = stock_data.json()
df = pd.DataFrame(data=json_data['data'],
                  columns=json_data['fields'])
df.tail()

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
13,113/07/18,"94,327,431","93,948,039,256",988.00,"1,005.00",986.00,"1,005.00",-25.00,"305,207"
14,113/07/19,"110,541,544","108,678,133,437",988.00,995.00,970.00,970.00,-35.00,"475,484"
15,113/07/22,"90,266,550","85,567,920,175",964.00,965.00,938.00,939.00,-31.00,"307,235"
16,113/07/23,"53,439,290","51,686,291,065",963.00,979.00,956.00,979.00,+40.00,"96,981"
17,113/07/26,"95,625,050","88,299,582,909",915.00,930.00,915.00,924.00,-55.00,"357,707"


### 3️⃣ 取得連續月份資料
以個股本益比為例

In [ ]:
# 設定抓取幾個月資料
month_num=3
date_now = dt.datetime.now()

# 建立日期串列
date_list = [(date_now - relativedelta(months=i)).replace(day=1).\
             strftime('%Y%m%d') for i in range(month_num)]

date_list.reverse()
all_df = pd.DataFrame()

# 使用迴圈抓取連續月份資料
for date in date_list:
  url = f'https://www.twse.com.tw/rwd/zh/afterTrading/\
      BWIBBU?date={date}&stockNo={stock_id}'
  try:
    json_data = requests.get(url).json()
    df = pd.DataFrame(data=json_data['data'],
                  columns=json_data['fields'])
    all_df = pd.concat([all_df, df], ignore_index=True)
  except Exception as e:
    print(f"無法取得{date}的資料, 可能資料量不足.")

all_df.head()

,日期,殖利率(%),股利年度,本益比,股價淨值比,財報年/季
0,113年05月02日,1.68,112,23.88,5.79,112/4
1,113年05月03日,1.67,112,24.13,5.85,112/4
2,113年05月06日,1.65,112,24.31,5.89,112/4
3,113年05月07日,1.63,112,24.74,6.00,112/4
4,113年05月08日,1.62,112,24.81,6.01,112/4


## 用 BeautifulSoup4 取得 Yahoo 股市資料

###4️⃣ 匯入套件

In [ ]:
from datetime import datetime
from bs4 import BeautifulSoup
import time

###  5️⃣ 取得當日股價

In [ ]:
def yahoo_stock(stock_id):
    url = f'https://tw.stock.yahoo.com/quote/{stock_id}.TW'
    # 使用 requests 取得網頁內容
    response = requests.get(url)
    html = response.content
    # 使用 Beautiful Soup 解析 HTML 內容
    soup = BeautifulSoup(html, 'html.parser')
    # 使用 find 與 find_all 定位元素
    time_element = soup.find('section',\
                {'id': 'qsp-overview-realtime-info'}).find('time')
    table_soups = soup.find('section',\
                {'id': 'qsp-overview-realtime-info'}).find('ul')\
                                   .find_all('li')
    fields = []
    datas = []
    for table_soup in table_soups:
        table_datas = table_soup.find_all('span')
        for num,table_data in enumerate(table_datas):
            if table_data.text =='':
                continue
            if num == 0:
                fields.append(table_data.text)
            else:
                datas.append(table_data.text)
    # 建立 DataFrame
    df = pd.DataFrame([datas], columns=fields)
    # 增加日期和股號欄位
    df.insert(0,'日期',time_element['datatime'])
    df.insert(1,'股號',stock_id)
    # 回傳 DataFrame
    return df

yahoo_stock(stock_id)

,日期,股號,成交,開盤,最高,最低,均價,成交金額(億),昨收,漲跌幅,漲跌,總量,昨量,振幅
0,2024/07/29 10:20,2330,943,942,948,939,942,181.89,924,2.06%,19.00,"19,304","81,991",0.97%


### 6️⃣ 取得季報表資訊


In [ ]:
# 函式可用於奇摩財報
def url_find(url):
    words = url.split('/')
    k = words[-1]
    # 使用requests取得網頁內容
    response = requests.get(url)
    html = response.content
    # 使用Beautiful Soup解析HTML內容
    soup = BeautifulSoup(html, 'html.parser')
    # 找到表格的表頭
    table_soup = soup.find('section', {'id': 'qsp-{}-table'.format(k)})
    table_fields=table_soup.find('div', class_='table-header')
    table_fields_lines = list(table_fields.stripped_strings)
    # 找到數據
    data_rows = table_soup.find_all('li' ,class_='List(n)')
    # 解析資料行內容
    data = []
    for row in data_rows:
        row_data = list(row.stripped_strings)
        data.append(row_data)
    # 建立 DataFrame
    df = pd.DataFrame(data, columns=table_fields_lines)
    return df

# 抓損益表
# url = f'https://tw.stock.yahoo.com/quote/{stock_id}/income-statement'
# 抓資產負債表
# url = f'https://tw.stock.yahoo.com/quote/{stock_id}/balance-sheet'
# 抓現金流量表
url = f'https://tw.stock.yahoo.com/quote/{stock_id}/cash-flow-statement'

# 抓取季報表資料
df = url_find(url).transpose()

# 資料處理
df.columns = df.iloc[0]
df = df[1:]
df.insert(0,'年度/季別',df.index)
df.columns.name = None
df.reset_index(drop=True, inplace=True)

df.tail()

,年度/季別,營業現金流,投資現金流,融資現金流,自由現金流,淨現金流
15,2020 Q2,"170,334,959","-129,750,164","5,912,053","40,584,795","36,836,214"
16,2020 Q1,"203,029,442","-188,993,268","-40,757,411","14,036,174","-24,622,107"
17,2019 Q4,"202,954,417","-171,605,106","-17,182,776","31,349,311","2,969,036"
18,2019 Q3,"141,753,021","-108,290,117","-229,112,243","33,462,904","-197,266,962"
19,2019 Q2,"117,761,028","-114,717,951","-930,502","3,043,077","4,026,735"


##使用 selenium 做新聞爬蟲

### 7️⃣ 用 requests 取得股票新聞

In [ ]:
# 股票代號
stock_id = "2330"
# 預設表格數據和欄位
field=['股號','日期','標題','內容']
data=[]
# 取得 Json 格式資料
json_data = requests.get(f'https://ess.api.cnyes.com/ess/api/'
                f'v1/news/keyword?q={stock_id}&limit=20&page=1').json()
# 依照格式擷取資料
items=json_data['data']['items']
for item in items:
    # 網址、標題和日期
    news_id = item["newsId"]
    title = item["title"]
    publish_at = item["publishAt"]
    # 使用 UTC 時間格式
    utc_time = datetime.utcfromtimestamp(publish_at)
    formatted_date = utc_time.strftime('%Y-%m-%d')
    # 前往網址擷取內容
    url = requests.get(f'https://news.cnyes.com/'
                       f'news/id/{news_id}').content
    soup = BeautifulSoup(url, 'html.parser')
    p_elements=soup .find_all('p')
    # 提取段落内容
    p=''
    for paragraph in p_elements[4:]:
        p+=paragraph.get_text()
    data.append([stock_id, formatted_date ,title,p])
# 建立表格
df = pd.DataFrame(data,columns=field)
df

,股號,日期,標題,內容
0,2330,2024-07-15,台積電:本公司代子公司 TSMC Global Ltd. 公告取得固定收益證券,2.交易日期:113/7/8~113/7/153.交易數量、每單位價格及交易總金額:6174...
1,2330,2024-07-11,新3千金展風華 3萬點有譜 華城1000元的秘密,值得一提的是，7 月 4 日不單是台積電股價站上千元關卡，同時華城及亞德客 - KY 股價也...
2,2330,2024-07-11,鉅亨速報 - Factset 最新調查：台積電(<mark>2330</mark>-TW)E...,歷史獲利表現詳細資訊請看台股內頁：https://www.cnyes.com/twstock...
3,2330,2024-07-11,鉅亨速報 - Factset 最新調查：台積電(<mark>2330</mark>-TW)目...,最新相關新聞資料來源：Factset，數據僅供參考，不作為投資建議。#波段上揚股51.43%...
4,2330,2024-07-10,"營收速報 - 加捷生醫(4109)6月營收7,186萬元年增率高達156.67％",近5日籌碼近6個月營收一覽 加捷生醫(4109-TW) 所屬產業為生技醫療業，主要業務...
5,2330,2024-07-10,"營收速報 - 台積電(<mark>2330</mark>)6月營收2,078.69億元年增...",近5日籌碼近6個月營收一覽 台積電(2330-TW) 所屬產業為半導體業，主要業務為依...
6,2330,2024-07-10,台積電:台積公司2024年6月營收報告,3.與公司關係(請輸入本公司或子公司):本公司4.相互持股比例:不適用5.發生緣由:不適用6...
7,2330,2024-07-08,機器人少不了他!! 機器視覺的關鍵技術就看這2家,阮蕙慈分析師認為，目前國內掛牌的影像擷取有 2 家。其中，圓剛 (2417-TW) 已經是全...
8,2330,2024-07-05,台股續寫新高！晶彩科、錸德攻上漲停？今日盤勢重點一次看,每日盤勢重點如何快速掌握？台股開盤日早上 8:30 準時鎖定「今天 Shot 這盤」延伸閱讀...
9,2330,2024-07-05,你不知道的AI飆股 智慧機械與機器人前世今生 生技AI的潛力,要體現 AI 的實質價值面，應該要回到 AI 科技的應用 (生產設備) 及應用 AI 所產出...


### 8️⃣  安裝及匯入套件

In [ ]:
!pip install selenium
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')  # 不顯示瀏覽器
chrome_options.add_argument('--no-sandbox')# 以最高權限運行

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.4 MB/s eta 0:00:00


### 9️⃣ 使用 Selenium 取得股票新聞

In [ ]:
# 透過 options 設定 driver
driver = webdriver.Chrome(options=chrome_options)
data2=[] # 表格數據
# 目標網址
url = f"https://www.cnyes.com/search/news?keyword={stock_id}"
driver.get(url)

# 模擬滑動滑鼠滾輪的行為，用於加載更多內容
scroll_pause_time = 2  # 等待時間
last_height = driver.execute_script(
                    "return document.body.scrollHeight")
while True:
    driver.execute_script(
        "window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)
    new_height = driver.execute_script(
        "return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

elements = driver.find_elements("xpath",
                        '//*[@id="_SearchAll"]/section/div/a')

# 擷取網址和標題
for element in elements:
    link = element.get_attribute("href")
    title = element.text
    title=title.split('\n')
    data2.append([stock_id, title[1] ,title[0],link])
# 關閉瀏覽器
driver.quit()
for link in data2:
  # 使用 requests 前往網址擷取新聞內容
  link_a = requests.get(link[3]).content
  link_b = BeautifulSoup(link_a,'html.parser')
  p_elements = link_b.find('article')
  # 取得段落内容
  try:
    link[3] = p_elements.get_text()
  except:
    link[3] = '無內容'
# 建立表格
df = pd.DataFrame(data2,columns=field)
df.tail()

,股號,日期,標題,內容
660,2330,2022/07/20,站上月線，別高興太早？,台股站上月線，別高興太早？韋國慶分析師(永誠投顧) 2022-07-20 20:20站上...
661,2330,2022/07/20,台積電:本公司代子公司 TSMC Global Ltd. 公告取得固定收益證券,公告台積電:本公司代子公司 TSMC Global Ltd. 公告取得固定收益證券鉅亨網新...
662,2330,2022/07/20,台積電:公告本公司百分之百持有之子公司TSMC Global Ltd.完成美金10億元無擔保...,公告台積電:公告本公司百分之百持有之子公司TSMC Global Ltd.完成美金10億元...
663,2330,2022/07/19,台積電:本公司代子公司 TSMC Global Ltd. 公告取得固定收益證券,公告台積電:本公司代子公司 TSMC Global Ltd. 公告取得固定收益證券鉅亨網新...
664,2330,2022/07/18,台積電:公告本公司111年度第4期無擔保普通公司債(綠色債券)主要發行條件,公告台積電:公告本公司111年度第4期無擔保普通公司債(綠色債券)主要發行條件鉅亨網新聞中...


```
#把資料存成 csv 可以用以下這段
df.to_csv('/content/news_data.csv' )
```

#3.3 用 Python 套件輕鬆取得股市資料

##使用 yfinance 下載股市資料

### 🔟  安裝及匯入套件

In [ ]:
!pip install yfinance
import yfinance as yf

### 1️⃣1️⃣  設定股票代碼和起止時間

In [ ]:
# 指定要下載的股票代碼, 上市為 .TW;上櫃為 .TWO
stock_id = '2330.TW'
# 設定開始與結束時間
end = dt.date.today()
start = end - dt.timedelta(days=360)

### 1️⃣2️⃣ 取得每日股價 (開高低收) 資料

In [ ]:
stock_data = yf.download(stock_id, start=start, end=end)
stock_data.tail()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-07-09,1030.0,1055.0,1025.0,1040.0,1040.0,46373121
2024-07-10,1020.0,1050.0,1015.0,1045.0,1045.0,46206302
2024-07-11,1065.0,1080.0,1055.0,1080.0,1080.0,41724336
2024-07-12,1030.0,1045.0,1025.0,1040.0,1040.0,68983062
2024-07-15,1040.0,1045.0,1025.0,1040.0,1040.0,38393309




```
# 依照資料期間下載
stock_data = yf.download(stock_id, period="3mo")

# 下載不同時間頻率的資料 (1分K)
stock_data = yf.download(stock_id, interval="1m")
```



### 1️⃣3️⃣ 取得多檔股票的資料

In [ ]:
stocks = [stock_id, '2303.TW', '2454.TW'] #分別為台積電、聯電和聯發科
stock_data = yf.download(stocks, start=start, end=end)
stock_data.tail()

[*********************100%%**********************]  3 of 3 completed


Price       Adj Close                      Close                       High  \
Ticker        2303.TW 2330.TW 2454.TW    2303.TW 2330.TW 2454.TW    2303.TW   
Date                                                                          
2024-07-09  55.299999  1040.0  1460.0  55.299999  1040.0  1460.0  56.299999   
2024-07-10  54.700001  1045.0  1435.0  54.700001  1045.0  1435.0  55.400002   
2024-07-11  54.700001  1080.0  1420.0  54.700001  1080.0  1420.0  55.200001   
2024-07-12  53.000000  1040.0  1360.0  53.000000  1040.0  1360.0  53.700001   
2024-07-15  53.400002  1040.0  1355.0  53.400002  1040.0  1355.0  53.599998   

Price                             Low                       Open          \
Ticker     2330.TW 2454.TW    2303.TW 2330.TW 2454.TW    2303.TW 2330.TW   
Date                                                                       
2024-07-09  1055.0  1460.0  55.000000  1025.0  1415.0  55.799999  1030.0   
2024-07-10  1050.0  1460.0  54.200001  1015.0  1410.0  55.200001  1020.0   
2024-07-11  1080.0  1460.0  54.200001  1055.0  1415.0  55.200001  1065.0   
2024-07-12  1045.0  1385.0  52.799999  1025.0  1350.0  53.700001  1030.0   
2024-07-15  1045.0  1380.0  53.000000  1025.0  1350.0  53.099998  1040.0   

Price                 Volume                     
Ticker     2454.TW   2303.TW   2330.TW  2454.TW  
Date                                             
2024-07-09  1450.0  83403492  46373121  9654123  
2024-07-10  1455.0  56944184  46206302  5443505  
2024-07-11  1435.0  60341856  41724336  5250273  
2024-07-12  1365.0  96626512  68983062  6871306  
2024-07-15  1370.0  46152642  38393309  3536752

### 1️⃣4️⃣ 取得公司基本資料


In [ ]:
stock = yf.Ticker(stock_id)
stock.info # 為字典型態

{'address1': 'Hsinchu Science Park',
 'address2': 'No. 8, Li-Hsin Road 6',
 'city': 'Hsinchu City',
 'zip': '300096',
 'country': 'Taiwan',
 'phone': '886 3 563 6688',
 'fax': '886 3 563 7000',
 'website': 'https://www.tsmc.com',
 'industry': 'Semiconductors',
 'industryKey': 'semiconductors',
 'industryDisp': 'Semiconductors',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Taiwan Semiconductor Manufacturing Company Limited, together with its subsidiaries, manufactures, packages, tests, and sells integrated circuits and other semiconductor devices in Taiwan, China, Europe, the Middle East, Africa, Japan, the United States, and internationally. It provides a range of wafer fabrication processes, including processes to manufacture complementary metal- oxide-semiconductor (CMOS) logic, mixed-signal, radio frequency, embedded memory, bipolar CMOS mixed-signal, and others. The company also offers customer and engineering support se

### 1️⃣5️⃣  取得損益表

In [ ]:
financials = stock.financials
financials.tail()

,2023-12-31,2022-12-31,2021-12-31,2020-12-31,2019-12-31
Other Gand A,60872800000.0,53524800000.0,36929600000.0,28457600000.0,NaN
Gross Profit,1175110600000.0,1348354800000.0,819537300000.0,711130100000.0,NaN
Cost Of Revenue,986625200000.0,915536500000.0,767877700000.0,628124700000.0,NaN
Total Revenue,2161735800000.0,2263891300000.0,1587415000000.0,1339254800000.0,NaN
Operating Revenue,2161735800000.0,2263891300000.0,1587415000000.0,1339254800000.0,NaN


In [ ]:
bs = stock.balance_sheet
bs

,2023-12-31,2022-12-31,2021-12-31,2020-12-31,2019-12-31
Treasury Shares Number,0.0,NaN,NaN,NaN,NaN
Ordinary Shares Number,25932070992.0,25930380458.0,25930380458.0,25930380458.0,NaN
Share Issued,25932070992.0,25930380458.0,25930380458.0,25930380458.0,NaN
Total Debt,956257900000.0,888174400000.0,753631900000.0,367792300000.0,NaN
Tangible Book Value,3406755700000.0,2877020500000.0,2122438100000.0,1809043200000.0,NaN
...,...,...,...,...,...
Cash Cash Equivalents And Short Term Investments,1714803200000.0,1586500100000.0,1204913700000.0,799893200000.0,NaN
Other Short Term Investments,249375400000.0,243686000000.0,139923500000.0,139722600000.0,NaN
Cash And Cash Equivalents,1465427800000.0,1342814100000.0,1064990200000.0,660170600000.0,NaN
Cash Equivalents,12326200000.0,13522700000.0,6182100000.0,6590000000.0,NaN


### 1️⃣6️⃣ 取得法人持股比例
因 yfinance 資料問題，此儲存格暫時無法使用

In [ ]:
institutional_holders = stock.institutional_holders
institutional_holders.tail()

YFDataException: Failed to parse holders json data.

## 使用 FinMind 下載股市資料

### 1️⃣7️⃣  安裝及匯入套件

In [ ]:
!pip install FinMind
from FinMind.data import DataLoader
import getpass

### 1️⃣8️⃣  輸入 FinMind API 和帳號密碼

In [ ]:
token = getpass.getpass("請輸入 FinMind 金鑰：")

請輸入 FinMind 金鑰：··········


### 1️⃣9️⃣  建立 FinMind 資料庫物件和登入 FinMind

In [ ]:
api = DataLoader()
api.login_by_token(api_token=token)

### 2️⃣0️⃣ 取得股價資料

In [ ]:
import datetime as dt # Import the datetime module and give it an alias dt
# 股票代號
stock_id = '2330'
# 資料期間
end = dt.date.today()
start = end - dt.timedelta(days=360)

stock_data =  api.taiwan_stock_daily(
    stock_id=stock_id,
    start_date=start,
    end_date=end)

stock_data.tail()

2024-07-16 13:09:28.312 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockPrice, data_id: 2330


,date,stock_id,Trading_Volume,Trading_money,open,max,min,close,spread,Trading_turnover
235,2024-07-10,2330,51810372,53308027550,1020.0,1050.0,1015.0,1045.0,5.0,61199
236,2024-07-11,2330,49304453,52782475514,1065.0,1080.0,1055.0,1080.0,35.0,65005
237,2024-07-12,2330,79472761,82222185037,1030.0,1045.0,1025.0,1040.0,-40.0,175230
238,2024-07-15,2330,44123104,45805242410,1040.0,1045.0,1025.0,1040.0,0.0,51387
239,2024-07-16,2330,36244442,38188299315,1040.0,1070.0,1035.0,1055.0,15.0,42202


### 2️⃣1️⃣ 取得損益表資料

In [ ]:
financial_data = api.taiwan_stock_financial_statement(
    stock_id=stock_id,
    start_date=str(start),)

financial_data.tail()

2024-07-16 13:10:04.624 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockFinancialStatements, data_id: 2330


,date,stock_id,type,value,origin_name
43,2024-03-31,2330,TotalNonoperatingIncomeAndExpense,1.752490e+10,營業外收入及支出
44,2024-03-31,2330,OtherComprehensiveIncome,4.127395e+10,其他綜合損益（淨額）
45,2024-03-31,2330,EquityAttributableToOwnersOfParent,2.673986e+11,綜合損益總額歸屬於母公司業主
46,2024-03-31,2330,NoncontrollingInterests,-9.033610e+08,綜合損益總額歸屬於非控制權益
47,2024-03-31,2330,GrossProfit,3.145053e+11,營業毛利（毛損）


### 2️⃣2️⃣ 取得法人買賣資料

In [ ]:
investors_data = api.taiwan_stock_institutional_investors(
    stock_id=stock_id,
    start_date=str(start),)
investors_data.tail()

2024-07-16 13:12:12.845 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockInstitutionalInvestorsBuySell, data_id: 2330


,date,stock_id,buy,name,sell
1195,2024-07-16,2330,0,Foreign_Dealer_Self,0
1196,2024-07-16,2330,248000,Dealer_self,743088
1197,2024-07-16,2330,172372,Dealer_Hedging,331315
1198,2024-07-16,2330,17084250,Foreign_Investor,16904472
1199,2024-07-16,2330,2356000,Investment_Trust,156500


## 使用 FinLab 下載股市資料

### 2️⃣3️⃣ 安裝及匯入套件

In [ ]:
!pip install finlab
import finlab
from finlab import data

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.1 MB/s eta 0:00:00
Mounted at /content/drive


### 2️⃣4️⃣ 登入 FinLab

In [ ]:
token = getpass.getpass("請輸入FinLab金鑰：")
finlab.login(token)

請輸入FinLab金鑰：··········
輸入成功!


### 2️⃣5️⃣ 取得收盤價

In [ ]:
close = data.get('price:收盤價')
close.tail()

Due to your status as a free user, the most recent data has been shortened or limited.
Daily usage: 18.9 / 500 MB - price:收盤價


symbol,0015,0050,0051,0052,0053,0054,0055,0056,0057,0058,...,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
date,,,,,,,,,,,,,,,,,,,,,
2020-12-25,NaN,118.95,42.83,106.50,55.85,27.57,17.69,29.51,78.55,NaN,...,22.20,40.60,16.10,11.9,10.05,88.5,19.95,112.5,NaN,9.73
2020-12-28,NaN,120.00,43.83,107.80,56.85,27.90,17.71,29.75,79.10,NaN,...,22.20,40.55,16.05,11.9,10.00,88.1,20.25,110.0,31.00,9.70
2020-12-29,NaN,119.90,43.79,108.00,56.80,27.90,17.71,29.67,79.60,NaN,...,22.25,40.60,16.05,11.9,9.97,87.8,19.85,109.0,34.10,9.70
2020-12-30,NaN,121.60,43.93,109.85,57.10,28.10,18.06,29.78,80.20,NaN,...,22.35,40.75,16.15,11.9,9.99,86.4,19.85,109.0,33.95,9.98
2020-12-31,NaN,122.25,44.00,110.20,57.70,28.21,18.05,29.95,80.40,NaN,...,22.50,40.90,16.15,11.9,10.15,91.3,19.75,107.0,34.00,9.98


### 2️⃣6️⃣ 選擇產業

In [ ]:
data.set_universe(market='TSE', category='半導體')
close = data.get('price:收盤價')
close.tail()

Daily usage: 18.9 / 500 MB - security_categories


symbol,2302,2303,2329,2330,2337,2338,2340,2342,2344,2351,...,6937,8016,8028,8081,8110,8131,8150,8162,8261,8271
date,,,,,,,,,,,,,,,,,,,,,
2020-12-25,24.50,46.75,14.90,511.0,40.40,40.35,25.4,36.95,26.70,77.0,...,NaN,159.0,56.9,158.5,14.05,37.45,34.40,NaN,53.0,39.35
2020-12-28,24.70,48.60,15.95,515.0,43.80,42.70,26.1,40.60,29.35,78.5,...,NaN,164.0,58.5,161.5,14.35,37.75,34.50,NaN,52.4,39.90
2020-12-29,23.80,47.40,15.50,515.0,41.85,41.50,25.9,39.50,28.40,86.3,...,NaN,161.0,58.2,159.5,13.90,37.70,35.00,NaN,51.3,39.45
2020-12-30,23.95,48.30,15.25,525.0,42.35,41.35,26.1,39.35,29.90,89.3,...,NaN,161.5,57.4,160.0,14.00,37.80,34.70,NaN,51.6,39.50
2020-12-31,24.50,47.15,15.05,530.0,42.30,40.35,27.5,38.65,29.05,98.2,...,NaN,162.5,58.7,159.5,13.80,37.80,34.35,NaN,52.9,39.30


### 2️⃣7️⃣ 取得財報資料

In [ ]:
df = data.get('financial_statement:每股盈餘')
df.tail()

Daily usage: 20.3 / 500 MB - financial_statement:每股盈餘


symbol,2302,2303,2329,2330,2337,2338,2340,2342,2344,2351,...,6937,8016,8028,8081,8110,8131,8150,8162,8261,8271
date,,,,,,,,,,,,,,,,,,,,,
2019-Q3,0.02,0.25,0.45,3.90,1.01,0.66,0.50,-1.10,0.15,0.69,...,NaN,3.22,0.83,2.50,0.41,0.80,0.81,NaN,0.22,1.34
2019-Q4,-0.08,0.32,0.08,4.48,0.41,0.62,0.38,-0.89,-0.05,0.31,...,NaN,2.54,0.44,2.19,0.32,0.69,0.72,NaN,0.11,0.99
2020-Q1,-0.09,0.19,0.02,4.51,0.67,-1.09,0.32,-0.69,-0.02,0.27,...,NaN,2.43,0.28,2.73,-0.27,0.88,0.98,NaN,0.29,1.53
2020-Q2,0.01,0.55,-0.50,4.66,0.72,0.66,0.19,-0.91,0.18,0.38,...,NaN,2.43,0.31,2.73,0.18,0.78,0.75,NaN,0.57,0.57
2020-Q3,0.21,0.75,0.02,5.30,0.88,1.27,0.55,2.00,0.08,0.51,...,NaN,2.63,0.11,3.55,0.42,0.80,0.58,NaN,0.68,0.24


### 2️⃣8️⃣  取得法人資料

In [ ]:
df = data.get('institutional_investors_trading_summary:投信買賣超股數')
df.tail()

Daily usage: 25.6 / 500 MB - institutional_investors_trading_summary:投信買賣超股數


symbol,2302,2303,2329,2330,2337,2338,2340,2342,2344,2351,...,6937,8016,8028,8081,8110,8131,8150,8162,8261,8271
date,,,,,,,,,,,,,,,,,,,,,
2020-12-25,0.0,1743000.0,0.0,-420000.0,-685000.0,0.0,0.0,250000.0,3566000.0,334000.0,...,NaN,8000.0,-122000.0,0.0,0.0,0.0,151000.0,NaN,0.0,0.0
2020-12-28,0.0,5804000.0,0.0,26000.0,816000.0,0.0,0.0,0.0,3847000.0,18000.0,...,NaN,64000.0,-200000.0,0.0,0.0,0.0,-142000.0,NaN,0.0,0.0
2020-12-29,0.0,2141930.0,0.0,-88180.0,1449000.0,0.0,0.0,100000.0,101000.0,1355000.0,...,NaN,-18000.0,-213000.0,0.0,0.0,0.0,2000.0,NaN,0.0,0.0
2020-12-30,0.0,452000.0,0.0,257000.0,3507000.0,0.0,0.0,0.0,5855000.0,1997000.0,...,NaN,27000.0,-104000.0,0.0,0.0,0.0,-90000.0,NaN,0.0,0.0
2020-12-31,0.0,263000.0,0.0,419000.0,396000.0,0.0,0.0,0.0,3237000.0,976000.0,...,NaN,-1000.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0


#3.4 SQL資料庫

### 2️⃣9️⃣ 匯入套件及連線資料庫

In [ ]:
import sqlite3
conn = sqlite3.connect('stock.db')

### 3️⃣0️⃣設定資料庫結構

In [ ]:
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS 日頻資料 (
    sno INTEGER PRIMARY KEY AUTOINCREMENT,
    Stock_Id TEXT,
    Date DATE,
    Open FLOAT,
    High FLOAT,
    Low FLOAT,
    Close FLOAT,
    Adj_Close FLOAT,
    Volume INTEGER
);
''')
conn.commit()

### 3️⃣1️⃣ 傳入資料到資料庫

In [ ]:
!pip install yfinance
import yfinance as yf

In [ ]:
df = yf.download('2330.TW',start='2023-08-01')
df = df.reset_index()
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
df.rename(columns={"Adj Close": "Adj_Close"}, inplace=True)
df.insert(0,'Stock_id','2330')

df.to_sql('日頻資料',conn,if_exists='append',index=False)

[*********************100%%**********************]  1 of 1 completed


234

### 3️⃣2️⃣ 查詢表格資料

In [ ]:
def table_info(table_name):
    cursor = conn.cursor()
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = cursor.fetchall()
    column_names = [column[1] for column in columns]
    print(f"資料庫表 '{table_name}' 的欄位名稱：", column_names)
    all_data = conn.execute(f'SELECT * FROM {table_name}')
    for i in all_data.fetchall():
        print(i)

# 查詢表格資料
table_info("日頻資料")

資料庫表 '日頻資料' 的欄位名稱： ['sno', 'Stock_Id', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj_Close', 'Volume']
(1, '2330', '2023-08-01', 565.0, 568.0, 564.0, 567.0, 556.16748046875, 16259643)
(2, '2330', '2023-08-02', 567.0, 569.0, 558.0, 561.0, 550.2821044921875, 25583234)
(3, '2330', '2023-08-04', 556.0, 560.0, 552.0, 554.0, 543.4158325195312, 26279173)
(4, '2330', '2023-08-07', 558.0, 561.0, 556.0, 558.0, 547.3394775390625, 14369551)
(5, '2330', '2023-08-08', 558.0, 558.0, 551.0, 552.0, 541.4541015625, 19010690)
(6, '2330', '2023-08-09', 550.0, 557.0, 550.0, 554.0, 543.4158325195312, 14389090)
(7, '2330', '2023-08-10', 552.0, 554.0, 550.0, 551.0, 540.4732055664062, 17892615)
(8, '2330', '2023-08-11', 556.0, 558.0, 546.0, 546.0, 535.5687255859375, 17608427)
(9, '2330', '2023-08-14', 540.0, 543.0, 537.0, 541.0, 530.6642456054688, 20939030)
(10, '2330', '2023-08-15', 543.0, 545.0, 540.0, 542.0, 531.6451416015625, 14066307)
(11, '2330', '2023-08-16', 541.0, 543.0, 536.0, 542.0, 531.6451416015625

### 3️⃣3️⃣ 新增資料

In [ ]:
def insert_data(stock_id, start):
  # 下載資料
  df = yf.download(f'{stock_id}.TW',start=start)
  df = df.reset_index()
  df.rename(columns={"Adj Close": "Adj_Close"}, inplace=True)
  df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
  df.insert(0,'Stock_Id',stock_id)

  # 新增資料表
  df.to_sql('日頻資料',conn,if_exists='append',index=False)

# 新增 2317 資料
insert_data(stock_id=2317, start='2023-08-01')

[*********************100%%**********************]  1 of 1 completed


### 3️⃣4️⃣ 從資料庫取得資料

In [ ]:
import pandas as pd # import pandas library and alias it as 'pd'

In [ ]:
query = ("SELECT Stock_id, Date, Close "
         "FROM 日頻資料 "
         "WHERE Date < '2023-08-15' AND Stock_id = '2317'")
df = pd.read_sql(query, conn, parse_dates=['Date'])
df.tail()

,Stock_Id,Date,Close
13,2317,2023-08-08,110.5
14,2317,2023-08-09,110.5
15,2317,2023-08-10,110.0
16,2317,2023-08-11,108.5
17,2317,2023-08-14,110.0


### 3️⃣5️⃣ 修改並關閉資料庫

In [ ]:
conn.commit()
conn.close()